**Sets directories:**
   - **anapyze_dir:** Directory where the anapyze_directory is located

In [3]:
import warnings
warnings.filterwarnings("ignore")

import os,sys
import shutil
import gzip
import nibabel as nib
from os.path import join, exists, isdir, basename
import numpy as np
import time


anapyze_dir = r'C:\Users\jesus\Work\repos\anapyze' 
anapyze_rsc = join(anapyze_dir,'resources')
sys.path.insert(0,anapyze_dir)
from spm import SPM

**Set directories:**
   - **Patient Data Directory (dir_patients):** This points to the directory where patient data is stored. *Note that this script expects that you have run the 0_Reorder_Data.py first*

**Set SPM:**
  
   - **SPM PATH:** Path for your installation of the Statistical Parametric Mapping (SPM) software 

**Template Images:**
   - **TPM (Tissue Probability Maps) Image (tpm):** This is set to the path where the TPM.nii file resides within the SPM software directory. **Check that this is correct**. 

   - **CAT12 Gray Scale Template Volume (template_volumes):** This is set to the path in the SPM directory where the CAT12 toolbox's template volumes are stored. **Check that this is correct**. 

In [6]:
# Put your data directory here
dir_patients = r'D:\IBIS_DATA\Prueba'

# Standalone SPM Paths
spm_path = r'C:\Users\jesus\Work\software\spm12'
#mcr_path = '/home/jsilva/software/Matlab_MCR/v93' 

# Change your templates here if necessary \toolbox\cat12\templates_MNI152NLin2009cAsym
tpm = join(spm_path, 'tpm','TPM.nii')
template_volumes = join(spm_path, r'toolbox\cat12\templates_MNI152NLin2009cAsym','Template_0_GS.nii')

This prepares a processing script that will process the collected T1 image files using SPM's CAT12 toolbox.

A MATLAB script named 'cat_12.m' is created that is recommended to be run separately in MATLAB. Alternatively you can run it here stating run=False in the *spm_proc.cat12seg_imgs* line, but you will not take advantage of the multi-threading capabilities of cat12. It is fine for a few images.

In [7]:
list_dirs = os.listdir(dir_patients)

#This will create a cat_12.m file that must be run on MATLAB separately

images = []

for i in list_dirs:

    dir_subj = join(dir_patients,i)
    check_cat_processing = [join(dir_subj, 'cat12', 'report', 'catreport_t1.pdf'),
                            join(dir_subj, 'cat12', 'mri', 'mwp1t1.nii'),
                            join(dir_subj, 'cat12', 'mri', 'mwp2t1.nii'),
                            join(dir_subj, 'cat12', 'mri', 'mwp3t1.nii'),
                            join(dir_subj, 'cat12', 'mri', 'p0t1.nii'),
                            join(dir_subj, 'cat12', 'mri', 'wmt1.nii')]

    if all(exists(j) for j in check_cat_processing):
        pass
    
    else:
    
        t1_image = False
        
        files_ = os.listdir(dir_subj)
    
        for file_ in files_:
    
            if file_[0:2] == 'IN':
                if file_[-3:] == 'nii':
                    if 'T1' in file_:
                        t1_image = join(dir_subj, file_)
                        
        if t1_image:
            
            cat12_dir = join(dir_subj, 'cat12')
            
            if not exists(cat12_dir):
                os.makedirs(cat12_dir)
    
            rm_in = join(cat12_dir,'t1.nii')
            shutil.copy(t1_image,rm_in)
    
            images.append(rm_in)

spm_proc = SPM(spm_path)
cat_12_proc = spm_proc.cat12seg_imgs(images, tpm, template_volumes, run=False)

*If you choose to run the previous step directly in MATLAB, you can run this in parallel*

**This cell performs preprocessing steps on Diffusion Tensor Imaging (DTI)** data for each subject in a given directory. The steps involve:

- Eddy current and movement correction via FSL (FreeSurfer)
- Denoising through DIPY's patch2self method
- Removal of Gibbs ringing artifacts.

The result is *dti_eddy_denoised.nii.gz* that is placed under the *dti* directory

In [ ]:
# DTI image preprocessing:
# Eddy current and movement correction with FSL
# Denoising using patch2self
# Gibbs ring artifact correction

from dipy.core.gradients import gradient_table
from dipy.io.gradients import read_bvals_bvecs
from dipy.denoise.patch2self import patch2self
from dipy.denoise.gibbs import gibbs_removal

list_dirs = os.listdir(dir_patients)

for i in list_dirs:
        
    print('\nProcessing %s: Subject %s/%s' %  (i, list_dirs.index(i), len(list_dirs)))
    start_time = time.time()

    dir_subj = join(dir_patients,i)

    dti_image = False
    bvec = False
    bval = False
    b_zero = False

    files_ = os.listdir(dir_subj)

    for file_ in files_:

        if file_[0:2] == 'IN':
            if file_[-4:] == 'bval':
                bval = join(dir_subj, file_)
            elif file_[-4:] == 'bvec':
                bvec = join(dir_subj, file_)
            elif file_[-3:] == 'nii':
                if 'DTI' in file_:
                    dti_image = join(dir_subj, file_)
                    if 'DTI001' in file_:
                        b_zero = 800
                    else:
                        b_zero = 1000

    if dti_image and bvec and bval and b_zero:

        dti_dir = join(dir_subj, 'dti')

        dti_out = join(dti_dir,'dti_eddy_denoised.nii.gz')
        
        if exists(dti_out):
            print("Result already exists!")
            b_file = join(dti_dir,str(b_zero))
            with open(b_file, 'w') as b0_file:
                pass
            pass
        
        else:
            
            # Check data 3D
            data_ = nib.load(dti_image).get_fdata()
            if len(data_.shape)!=4:
                print("Data shape:" ,  data_.shape, ". Something is wrong with the shape of the data. Ignoring this image...")
                pass
            
            else:
                print("Data shape:" ,  data_.shape)

                if not exists(dti_dir):
                    os.makedirs(dti_dir)
        
                dti_in = join(dti_dir,'dti.nii')
                shutil.copy(dti_image,dti_in)
                
                bval_in = join(dti_dir,'dti.bval')
                shutil.copy(bval,bval_in)
        
                bvec_in = join(dti_dir,'dti.bvec')
                shutil.copy(bvec,bvec_in)
        
                bvals, bvecs = read_bvals_bvecs(bval_in, bvec_in)
                gtab = gradient_table(bvals, bvecs)
        
                dti_compressed = join(dti_dir,'dti_eddy.nii.gz')
        
                # Image Corrections
                print("Correcting DTI for eddy currents....")
                log_file = join(dti_dir,'eddy_correct.log')
                os.system('eddy_correct %s %s 0 spline >> %s' % (dti_in, dti_compressed, log_file))
                
                dti_img = nib.load(dti_compressed)
                dti_data = dti_img.get_fdata()
                hdr = dti_img.header
                affine = dti_img.affine
                
                print("Denoising....")
                denoised_arr = patch2self(dti_data, bvals, model='ols', shift_intensity=True, clip_negative_vals=False, b0_threshold=50)
                
                print("Removing Gibbs artifacts....")
                gibbs_corr = gibbs_removal(denoised_arr, slice_axis=2, num_processes=-1)
        
                img = nib.Nifti1Image(gibbs_corr, affine, hdr)
                nib.save(img, dti_out)
                
                b_file = join(dti_dir,str(b_zero))
                with open(b_file, 'w') as b0_file:
                    pass

                print("I finished Processing %s: It took me %s minutes " % (i, (time.time() - start_time) / 60))
                
                


Processing 1080: Subject 0/221
Result already exists!

Processing 11409: Subject 1/221
Result already exists!

Processing 11446: Subject 2/221
Result already exists!

Processing 11453: Subject 3/221
Result already exists!

Processing 11460: Subject 4/221
Result already exists!

Processing 11477: Subject 5/221
Result already exists!

Processing 11495: Subject 6/221
Result already exists!

Processing 11627: Subject 7/221
Result already exists!

Processing 11731: Subject 8/221

Processing 11732: Subject 9/221
Result already exists!

Processing 11766: Subject 10/221
Result already exists!

Processing 11778: Subject 11/221
Result already exists!

Processing 11782: Subject 12/221
Result already exists!

Processing 11797: Subject 13/221
Result already exists!

Processing 11841: Subject 14/221
Result already exists!

Processing 11862: Subject 15/221
Result already exists!

Processing 11879: Subject 16/221
Result already exists!

Processing 11904: Subject 17/221
Result already exists!

Process

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


This cell aims to correct distortions in the DTI dataset of each patient using 'dipy', 'nilearn', and 'nibabel' libraries along with ANTs software. 

*This needs the inputs from both previous cells (including the cat12 results) so wait those to finish*

In [ ]:
# Registration-based distortion-correction:

import nilearn.image as proc

#Processing patients
list_dirs = os.listdir(dir_patients)


for i in list_dirs:

    output_file = join(dir_patients, i, 'dti', 'dti_ants.nii.gz')

    if exists(output_file):
        print('Patient %s is done' % i)
        pass
    
    else:
        
        dir_subj = join(dir_patients,i)
        print('\nProcessing %s: Subject %s/%s' %  (i, list_dirs.index(i), len(list_dirs)))
        start_time = time.time()
    
        dir_t1 = join(dir_subj, 'cat12','mri')
        dir_dti = join(dir_subj,'dti')
    
        t1_source = join(dir_t1, 'p0t1.nii')
        dti_source = join(dir_dti, 'dti_eddy_denoised.nii.gz')
        
        if exists(t1_source) and exists(dti_source):
    
            t1_img = nib.load(t1_source)
            t1_data = t1_img.get_fdata()
            t1_data = t1_data.astype(float)
            t1_img.set_data_dtype(float)
        
            indx = np.where(t1_data > 0)
            t1_data[indx] = 1 / (t1_data[indx])
        
            inverted_t1 = nib.Nifti1Image(t1_data, t1_img.affine, t1_img.header)
            inverted_t1 = proc.smooth_img(inverted_t1, 2)
            inverted_name = join(dir_t1, 't1_inverted.nii.gz')
            nib.save(inverted_t1, inverted_name)
        
            ants_log = 'ANTs_log.txt'
        
            img_4d = nib.load(dti_source)
            # Get the 4D data array
            data_4d = img_4d.get_fdata()
            # Extract the first frame (3D) from the 4D data
            data_3d = data_4d[:, :, :, 0]
        
            # Create a new 3D NIfTI image with the same header as the original 4D image
            img_3d = nib.Nifti1Image(data_3d, img_4d.affine, img_4d.header)
            b0 = join(dir_dti, 'b0.nii.gz')
            nib.save(img_3d, b0)
        
            log = join(dir_subj, 'ants.log')
        
            print("Corregistering T1 with b0....")
        
            command = 'antsRegistrationSyN.sh -d 3 -f %s -m %s -o %s/t1 -t r -n 6 > %s' % (b0, inverted_name, dir_t1, log)
            os.system(command)
        
            print("Deforming b0....")
        
            t1_warped = join(dir_t1,'t1Warped.nii.gz')
        
            command = 'antsRegistrationSyN.sh -d 3 -f %s -m %s -o %s/dti -t s -n 12 >> %s' % (t1_warped, b0, dir_dti, log)
            os.system(command)
        
            dti_warped = join(dir_dti,'dtiWarped.nii.gz')
            dti_warp = join(dir_dti,'dti1Warp.nii.gz')
        
            warped_files = []
        
            print("Applying the transformation to the whole dataset....")
        
            for k in range(data_4d.shape[3]):
                data_3d = data_4d[:, :, :, k]
                img_3d = nib.Nifti1Image(data_3d, img_4d.affine, img_4d.header)
                bk = join(dir_dti, 'temp_b%s.nii.gz' % k)
                nib.save(img_3d, bk)
                warped_bk = join(dir_dti, 'warped_b%s.nii.gz' % k)
        
                command = 'antsApplyTransforms -d 3 -i %s -r %s -o %s -t %s >> %s' % (bk, dti_warped, warped_bk, dti_warp, log)
                os.system(command)
        
                warped_files.append(warped_bk)
        
            data_3d_list = [nib.load(file).get_fdata() for file in warped_files]
        
            data_4d = np.stack(data_3d_list, axis=3)
        
            affine = nib.load(warped_files[0]).affine
            header = nib.load(warped_files[0]).header
            header.set_data_shape(data_4d.shape)
        
            img_4d = nib.Nifti1Image(data_4d, affine, header)
        
            nib.save(img_4d, output_file)
        
            warped_t1 = join(dir_t1, 't1Warped.nii.gz')
        
            to_remove = join(dir_dti, 'temp_*')
            os.system('rm %s' % to_remove)
            to_remove = join(dir_dti, 'warped_*')
            os.system('rm %s' % to_remove)
        
            print("I finished Processing %s: It took me %s minutes " % (basename(dir_subj), (time.time() - start_time) / 60))
        
        else:
            print("Images missing")


This script applies a brain mask to the 4D DTI images already preprocessed and corrected for distortion for each patient.
**This needs the input data from the previous cell**

In [ ]:
#Now we will mask data

list_dirs = os.listdir(dir_patients)

for i in list_dirs:

    print('\nProcessing %s: Subject %s/%s' %  (i, list_dirs.index(i), len(list_dirs)))
    start_time = time.time()

    dir_subj = join(dir_patients,i)
    dir_dti = join(dir_subj,'dti')

    in_file = join(dir_dti, 'dti_ants.nii.gz')

    out_mask = join(dir_dti, 'dti_mask.nii.gz')
    out_masked = join(dir_dti, 'dti_masked.nii.gz')

    if exists(in_file) and not exists(out_masked):

        out = join(dir_dti, 'dti')
        os.system('bet %s %s -m -f 0.2 -n' % (in_file, out))
        
        if exists(out_mask):

            img_in = nib.load(in_file)
            mask_in = nib.load(out_mask)

            img_data = img_in.get_fdata()
            mask_data = mask_in.get_fdata()
            
            mask_data = np.expand_dims(mask_data, axis=3)
            masked_data = img_data * mask_data

            img = nib.Nifti1Image(masked_data, img_in.affine, img_in.header)
            nib.save(img, out_masked)

        print("I finished Processing %s: It took me %s minutes " % (i, (time.time() - start_time) / 60))

    else:
        print("I skipped this one. Already processed?")


Now you are ready to run the **Process_Cohort** script in the fw_pasternak directory.
------------------------------------------------------------------------------------------

After Process_Cohort is done we can do some post-processing.

In [ ]:
list_dirs = os.listdir(dir_patients)

client = docker.from_env()

for i in list_dirs:

    dir_subj = join(dir_patients,i)
    dir_dti = join(dir_subj,'dti')
    dir_t1 = join(dir_subj,'cat12','mri')
    
    
    print('\nPreparing postprocessing dirs %s: Subject %s/%s' %  (i, list_dirs.index(i), len(list_dirs)))

    results_dipy = ['dti_md.nii.gz', 'dti_fa.nii.gz']
    results_pasternak = ['fw_FW.nii.gz','fw_NoNeg_MD.nii.gz', 'fw_NoNeg_FA.nii.gz', 
                         'fw_FWE.nii.gz', 'fw_FWE_MD.nii.gz', 'fw_FWE_FA.nii.gz']

    for i in results_dipy:
        
        result = join(dir_dti,i)
        warp = join(dir_t1,'t10GenericAffine.mat')
        t1_inverted = join(dir_t1,'t1_inverted.nii.gz')
        out_ants = join(dir_dti,'t1_%s' % i)

        if exists(result):

            command='antsApplyTransforms -d 3 -i %s -r %s -o %s -t [%s,1]' % (result, t1_inverted, out_ants, warp)
            os.system(command)
    
            with gzip.open(out_ants, 'rb') as f_in:
                with open(out_ants[0:-3], 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
                    
            os.remove(out_ants)
        
        else:
            print('Some file missing')

    img_to_deform = []
    
    for i in results_dipy:
        
        out_ants = 't1_%s' % i[0:-3]
        in_docker = join('/dti_data',out_ants)
        
        if exists(join(dir_dti,out_ants)):
            img_to_deform.append(in_docker)

    if img_to_deform:

        def_matrix = join('/mri_data', 'y_t1.nii')
    
        mfile_name = join(dir_dti, 'deformations.m')
    
        design_type_comp = "matlabbatch{1}.spm.util.defs.comp{1}."
        design_type_out = "matlabbatch{1}.spm.util.defs.out{1}."
    
        new_spm = open(mfile_name, "w")
    
        new_spm.write(
                design_type_comp + "def = {'" + '/mri_data/y_t1.nii' + "'};\n" +
                design_type_out + "pull.fnames = {" + "\n")
    
        for image in img_to_deform:
            new_spm.write("'" + image + "'\n")
        new_spm.write("};\n")
    
        new_spm.write(
            design_type_out + "pull.savedir.savesrc = 1;\n" +
            design_type_out + "pull.interp =" + str(4) + ";\n" +
            design_type_out + "pull.mask = 0;\n" +
            design_type_out + "pull.fwhm = [0 0 0];\n" +
            design_type_out + "pull.prefix ='" + 'w' + "';\n"
            )
    
        new_spm.close()
    
        client.containers.run(image="jhuguetn/cat12", auto_remove=True, tty=True, stdin_open=True,
                                volumes={dir_dti: {"bind": "/dti_data", "mode": "rw"},
                                        dir_t1: {"bind": "/mri_data", "mode": "rw"}},
                                command="-b /dti_data/deformations.m")


    for i in results_pasternak:
        
        result = join(dir_dti,'Pasternak',i)
        warp = join(dir_t1,'t10GenericAffine.mat')
        t1_inverted = join(dir_t1,'t1_inverted.nii.gz')
        out_ants = join(dir_dti,'t1_%s' % i)

        if exists(result):

            command='antsApplyTransforms -d 3 -i %s -r %s -o %s -t [%s,1]' % (result, t1_inverted, out_ants, warp)
            os.system(command)
    
            with gzip.open(out_ants, 'rb') as f_in:
                with open(out_ants[0:-3], 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
                    
            os.remove(out_ants)
        
        else:
            print('Some file missing')

    img_to_deform = []
    
    for i in results_pasternak:
        
        out_ants = 't1_%s' % i[0:-3]
        in_docker = join('/dti_data',out_ants)
        
        if exists(join(dir_dti,out_ants)):
            img_to_deform.append(in_docker)

    if img_to_deform:

        def_matrix = join('/mri_data', 'y_t1.nii')
    
        mfile_name = join(dir_dti, 'deformations.m')
    
        design_type_comp = "matlabbatch{1}.spm.util.defs.comp{1}."
        design_type_out = "matlabbatch{1}.spm.util.defs.out{1}."
    
        new_spm = open(mfile_name, "w")
    
        new_spm.write(
                design_type_comp + "def = {'" + '/mri_data/y_t1.nii' + "'};\n" +
                design_type_out + "pull.fnames = {" + "\n")
    
        for image in img_to_deform:
            new_spm.write("'" + image + "'\n")
        new_spm.write("};\n")
    
        new_spm.write(
            design_type_out + "pull.savedir.savesrc = 1;\n" +
            design_type_out + "pull.interp =" + str(4) + ";\n" +
            design_type_out + "pull.mask = 0;\n" +
            design_type_out + "pull.fwhm = [0 0 0];\n" +
            design_type_out + "pull.prefix ='" + 'w' + "';\n"
            )
    
        new_spm.close()
    
        client.containers.run(image="jhuguetn/cat12", auto_remove=True, tty=True, stdin_open=True,
                                volumes={dir_dti: {"bind": "/dti_data", "mode": "rw"},
                                        dir_t1: {"bind": "/mri_data", "mode": "rw"}},
                                command="-b /dti_data/deformations.m")

    os.system('rm %s/t1_*' % dir_dti)
    
    